# Visualization of DPS results in a map

In [1]:
!pip install cogeo-mosaic

In [2]:
from glob import glob
import os
import json
import urllib
from typing import Dict

import folium
import geopandas as gpd
from cogeo_mosaic.mosaic import MosaicJSON
from cogeo_mosaic.backends import MosaicBackend

/projects/.local/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [3]:
def local_to_s3(url):
    return url.replace("/projects/my-private-bucket", "s3://maap-ops-workspace/nathanmthomas")
    
def s3_to_https(url):
    parts = url.split("/")
    return os.path.join('https://',f'{parts[2]}.s3.amazonaws.com', *parts[3:])

In [4]:
dps_files = '/projects/tile-index2.shp'
dps_output = gpd.read_file(dps_files)
dps_output.head()

DriverError: /projects/tile-index2.shp: No such file or directory

In [ ]:
dps_output['s3'] = local_to_s3(dps_output['location'])

In [ ]:
tile_index_path = '/projects/shared-buckets/nathanmthomas/boreal_grid_albers90k_gpkg.gpkg'
tile_index = gpd.read_file(tile_index_path)

In [ ]:
albers = tile_index.crs
dps_output.set_crs(albers, inplace=True)

In [ ]:
dps_output_4236 = dps_output.to_crs("EPSG:4326")

In [ ]:
m = folium.Map()
folium.GeoJson(dps_output_4236).add_to(m)
m

## Create MosaicJson

In [ ]:
def get_accessor(feature: Dict):
    """Return specific feature identifier."""
    return feature["properties"]["s3"]



In [ ]:
geojson = json.loads(dps_output_4236.to_json())

In [ ]:
mosaicdata = MosaicJSON.from_features(geojson.get('features'), minzoom=8, maxzoom=16, accessor=get_accessor)

In [ ]:
mosaic_json = "/projects/my-public-bucket/elev_check.json"
with MosaicBackend(mosaic_json, mosaic_def=mosaicdata) as mosaic:
    mosaic.write(overwrite=True)



## See it on a map

In [ ]:
wmts_url = "https://h9su0upami.execute-api.us-east-1.amazonaws.com/mosaicjson/WMTSCapabilities.xml"
params = {
    "tile_format":"png",
    "tile_scale":"1",
    "pixel_selection":"first",
    "TileMatrixSetId":"WebMercatorQuad",
    "url":"https://maap-ops-workspace.s3.amazonaws.com/shared/nathanmthomas/elev_check.json",
    "bidx":"3", # Select which band to use
    "resampling_method":"nearest",
    "rescale":"0,1", # Values in data are from 0 to 1
    "return_mask":"true",
    "colormap_name":"magma" # Any colormap from matplotlib will work
}

wmts_call = "?".join([wmts_url, urllib.parse.urlencode(params)])

# Note Jupyter bug add amp; incorrectly when printing the url
wmts_call

In [ ]:
import ipycmc
w = ipycmc.MapCMC()
w

In [ ]:
# This adds a new layer to the map above, call Cloud Optimized GeoTIFF
w.load_layer_config(wmts_call, "wmts/xml")

